In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web

In [2]:
momentos = lambda x,c: ((x - x.mean())**c).mean() / (x.std()**c)

In [3]:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

In [4]:
SP500 = pd.read_csv('SP500.csv',header='infer',encoding= 'unicode_escape')

In [5]:
list = SP500[SP500.iloc[:,1]=='Communication Services'].iloc[:,0].to_numpy()

In [6]:
closes = get_adj_closes(list, "01-01-2015",'31-12-2015')

C:\Users\flavi\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'FOXA', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\flavi\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'FOX', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [27]:
closes1 = get_adj_closes(['EWW','EWG','EWY','EWJ','EWH','EWZ','KSA','ERUS','TUR','EWL'], "01-01-2016",'06-01-2016')

In [30]:
closes2 = get_adj_closes(['EWW','EWG','EWY','EWJ','EWH','EWZ','KSA','ERUS','TUR','EWL'], "06-02-2016",'31-12-2016')

In [31]:
rend1 = closes1.pct_change().dropna()
rend2 = closes2.pct_change().dropna()
c1 =rend1.corr()
c2 = rend2.corr()

In [34]:
M = np.array([c1,c2])

In [7]:
closes.dropna(axis = 1, inplace = True)

In [8]:
rend = closes.pct_change().dropna()

In [9]:
info = pd.DataFrame({'Mean':rend.mean(), 'Std':rend.std()})

In [10]:
assy = [momentos(rend.iloc[:,i],3) for i in range(len(list)-2)]

In [19]:
kurt = [momentos(rend.iloc[:,i],4) for i in range(len(list)-2)]

In [20]:
info['Kurt'] = kurt

In [11]:
info['Assy'] = assy

In [12]:
x = info.Mean.to_numpy()

In [13]:
scaler = lambda x: (x - x.min())/(x.max()-x.min())

In [14]:
info['std2'] = scaler(info.Std.to_numpy())
info['mean2'] = scaler(info.Mean.to_numpy())
info['assy2'] = scaler(info.Assy.to_numpy())

In [23]:
import plotly.express as px
fig = px.scatter_3d(info, x='Std', y='Mean', z='Assy', color = 'Kurt')
fig.show()

fig2 = px.scatter_3d(info, x='std2', y='mean2', z='assy2', color = 'Kurt')
fig2.show()

In [56]:
x = np.linspace(0, 1, 70)
X, Y = np.meshgrid(x, x)
Z = np.sin(X**(1/7)+np.pi/2)+np.sin(Y**(1/7)+np.pi/2)-1

In [105]:
Z =np.sin(X**2+np.pi/2)*np.cos(Y**2-np.pi/2)

In [69]:
d = lambda x,y,z: np.sin(x**2+np.pi/2)*np.cos(y**2-np.pi/2) -z

In [113]:
info['new_metric'] = d(-info.mean2+1,info.assy2,info.std2)

In [115]:
layout = go.Layout(width = 700, height =700,
                             title_text='Chasing global Minima')
fig = go.Figure(data=[go.Surface(x = X, y = Y, z=Z,colorscale = 'Blues')], layout=layout)
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))

#fig.add_scatter3d(x=X.flatten(), y=Y.flatten(), z = Z. flatten(), mode='markers', 
#                  marker=dict(size=2, color=X.flatten(),               
#                              colorscale='Reds'))
fig.add_scatter3d(z=info.std2, x=-info.mean2+1, y = info.assy2, mode='markers', 
                  marker=dict(size=3,color= info.new_metric))

In [80]:
info['Norm']=-2*scaler(info.Std.to_numpy())+2*scaler(info.Mean.to_numpy())+3*scaler(info.Assy.to_numpy())

In [58]:
a = info.sort_values(by = 'Assy').index[-3:].to_numpy()
b = info.sort_values(by = 'Mean').index[-3:].to_numpy()
c = info.sort_values(by = 'Std').index[:3].to_numpy()

In [81]:
g = info.sort_values(by = 'Norm').index[-9:].to_numpy()

In [124]:
g =info.sort_values(by = 'new_metric',ascending = False).index[:9].to_numpy()

In [125]:
g

array(['GOOGL', 'GOOG', 'TTWO', 'T', 'VZ', 'ATVI', 'OMC', 'LYV', 'CMCSA'],
      dtype=object)

In [126]:
info.sort_values(by = 'new_metric', ascending = False)

,Mean,Std,Assy,std2,mean2,assy2,Kurt,new metric,new_metric,Norm
Symbols,,,,,,,,,,
GOOGL,0.001693,0.018213,3.001723,0.375347,0.594132,0.993338,27.058769,-0.624612,0.447582,3.417583
GOOG,0.001648,0.018620,2.943925,0.393853,0.585416,0.983017,24.717387,-0.605875,0.416825,3.332178
TTWO,0.001046,0.019852,3.039035,0.449769,0.469676,1.000000,29.469145,-0.550231,0.358641,3.039814
T,0.000334,0.009943,0.455024,0.000000,0.332865,0.538604,4.298720,-0.790014,0.258177,2.281542
VZ,0.000176,0.009949,0.404401,0.000271,0.302315,0.529565,4.268630,-0.781138,0.244359,2.192782
ATVI,0.002800,0.017541,1.435891,0.344859,0.806814,0.713746,12.526490,-0.590028,0.142483,3.065147
OMC,0.000153,0.010880,0.002695,0.042542,0.298069,0.457837,2.866926,-0.668875,0.140791,1.884566
LYV,-0.000151,0.014155,0.687162,0.191170,0.239618,0.580054,6.886373,-0.633678,0.085317,1.837058
CMCSA,0.000087,0.013020,-0.306025,0.139658,0.285299,0.402713,3.597890,-0.512265,0.001199,1.499419


In [73]:
f =np.append(np.append(a,b),c)

In [15]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [47]:
N = 2

fig = go.Figure(
    data=[go.Heatmap(z=M[0],colorscale='RdBu')],
    layout=go.Layout(
        title="Frame 0",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 2000, "redraw": True},}])])]
    ),
    frames=[go.Frame(data=[go.Heatmap(z=M[i], colorscale='RdBu')],
                     layout=go.Layout(title_text=f"Frame {i}")) 
            for i in range(1, N)]
)
fig.show()

In [45]:
go.Heatmap?

In [18]:
c = rend.corr()
fig = px.imshow(c, color_continuous_scale='RdBu',template='plotly_dark')
fig.update_layout(title='Heat Map for Correlation Matrix')
fig.show()

In [61]:
fig = make_subplots(
    rows=3, cols=3,specs=[[{"type": "barpolar"}, {"type": "barpolar"},{"type": "barpolar"}],
           [{"type": "barpolar"}, {"type": "barpolar"},{"type": "barpolar"}],
                         [{"type": "barpolar"}, {"type": "barpolar"},{"type": "barpolar"}],],shared_yaxes= 'all',
    shared_xaxes='all')
for i in range(9):
    y = info[info.index == f[i]]
    fig.add_trace(go.Barpolar(name =f[i] ,theta=['Mean','std','Assy'],
                              r=info[info.index == f[i]].iloc[0,:3]*np.array([100,10,0.1])),row=i//3+1, col=i%3+1 )

fig.update_layout(template='plotly_dark', title = 'Four Statistical Moments for every Asset')
fig.show()

In [127]:
fig = make_subplots(
    rows=3, cols=3,specs=[[{"type": "barpolar"}, {"type": "barpolar"},{"type": "barpolar"}],
           [{"type": "barpolar"}, {"type": "barpolar"},{"type": "barpolar"}],
                         [{"type": "barpolar"}, {"type": "barpolar"},{"type": "barpolar"}],],shared_yaxes= 'all',
    shared_xaxes='all')
for i in range(9):
    y = info[info.index == g[i]]
    fig.add_trace(go.Barpolar(name =g[i] ,theta=['Mean','std','Assy'],
                              r=info[info.index == g[i]].iloc[0,:3]*np.array([100,10,0.1])),row=i//3+1, col=i%3+1 )

fig.update_layout(template='plotly_dark', title = 'Four Statistical Moments for every Asset')
fig.show()